In [ ]:
import os, pickle
%run CollectForestInfo.ipynb
%run RasMMA.ipynb

In [ ]:
base_dir = 'output/top3_party_0622/'
tag = '28fam_cross'
outputPath = 'output/begC/'+tag+'/'

famNames = os.listdir(base_dir)
cand_clusters = dict()
candCounter = 0
for fName in famNames:
    pickleDir = base_dir+fName+'/pickle/'
    interPkl = pickleDir + fName + "_intermediate.pickle"
    residualPkl = pickleDir + fName + "_residual.pickle"
    forestInfo = CollectForestInfo(interPkl,
                           residualPkl,
                           True) # one pickle is a forest

    for treeName in forestInfo.getTreeRootNameList():
        labelName = fName+'_'+treeName
        memberCount = len(forestInfo.getTreeMembers(treeName))
        repSeq = forestInfo.getRepAPISeq(treeName)
        if memberCount > 2 and len(repSeq) > 10:
            clusterName = 'G'+str(candCounter)
            R = (clusterName, [repSeq])
#             R = (clusterName, [(repSeq, 0, len(repSeq)-1)] )
            clusterMembers = set()
            clusterMembers.add(labelName)
            cand_clusters[candCounter] = (R, clusterMembers)
            candCounter += 1

In [ ]:
threshold = 0.8
intermediatePool, initialDict, roundInfos, residual = clusterInitializedReps(cand_clusters,
                                                                             tag,
                                                                             outputPath,
                                                                             threshold)
pickleDir = outputPath + 'pickle/'
if not os.path.isdir(pickleDir): os.makedirs(pickleDir)

# saving intermediatePool as pickle file
with open(pickleDir + tag + '_intermediate.pickle', 'wb') as handle:
    pickle.dump(intermediatePool, handle, protocol=pickle.HIGHEST_PROTOCOL)

# saving initialNames dict as pickle file
with open(pickleDir + tag + '_initialDict.pickle', 'wb') as handle:
    pickle.dump(initialDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# saving round information dict as pickle file
with open(pickleDir + tag + '_roundInfos.pickle', 'wb') as handle:
    pickle.dump(roundInfos, handle, protocol=pickle.HIGHEST_PROTOCOL)

if(residual is not None):
    # saving round information dict as pickle file
    with open(pickleDir + tag + '_residual.pickle', 'wb') as handle:
        pickle.dump(residual, handle, protocol=pickle.HIGHEST_PROTOCOL)